In [ ]:
df1=df.copy()

In [2]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder , RobustScaler
from category_encoders import BinaryEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import make_scorer, r2_score, mean_squared_error

In [3]:
df = pd.read_csv("EDA_OUT.csv")
df

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_Month,Journey_DayOfWeek
0,IndiGo,Banglore,New Delhi,22,1,2.833333,0,no info,3897,3,Sunday
1,Air India,Kolkata,Banglore,5,13,7.416667,2,no info,7662,5,Wednesday
2,Jet Airways,Delhi,Cochin,9,4,19.000000,2,no info,13882,6,Sunday
3,IndiGo,Kolkata,Banglore,18,23,5.416667,1,no info,6218,5,Sunday
4,IndiGo,Banglore,New Delhi,16,21,4.750000,1,no info,13302,3,Friday
...,...,...,...,...,...,...,...,...,...,...,...
10457,Air Asia,Kolkata,Banglore,19,22,2.500000,0,no info,4107,4,Tuesday
10458,Air India,Kolkata,Banglore,20,23,2.583333,0,no info,4145,4,Saturday
10459,Jet Airways,Banglore,Delhi,8,11,3.000000,0,no info,7229,4,Saturday
10460,Vistara,Banglore,New Delhi,11,14,2.666667,0,no info,12648,3,Friday


In [8]:
X = df.drop("Price" , axis = 1 )
y = df["Price"]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
Encoder = ColumnTransformer(transformers=[("OHE",OneHotEncoder(sparse_output=False , drop="first" ), 
                                           ["Journey_DayOfWeek" , "Source","Destination"]) , ("BE",BinaryEncoder() , 
                                                                                              ["Airline" , "Additional_Info"] )] , remainder = "passthrough")

In [16]:
steps = []
steps.append(("Encoder" , Encoder))
steps.append(("Scaler" , RobustScaler()))
steps.append(("Model" , LogisticRegression()))
pipeline = Pipeline(steps=steps)

In [19]:
scoring = {'r2': make_scorer(r2_score), 'mse': make_scorer(mean_squared_error)}
logreg = LogisticRegression(max_iter=1000)

In [20]:
results = cross_validate(pipeline, X, y, cv=5, scoring=scoring, return_train_score=True)

C:\Users\LENOVO\anaconda3\envs\train\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\LENOVO\anaconda3\envs\train\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\LENOVO\anaconda3\envs\train\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown

In [21]:
train_r2_mean = results["train_r2"].mean()
print("Mean Training R2 Score:", train_r2_mean)

Mean Training R2 Score: 0.5648062769983571


In [22]:
test_r2_mean = results["test_r2"].mean()
print("Mean Test R2 Score:", test_r2_mean)

Mean Test R2 Score: 0.5533568556202276


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb


In [30]:
models = list()
models.append(("RF" , RandomForestRegressor()))
models.append(("GB" , GradientBoostingRegressor()))
models.append(("LG" , lgb.LGBMRegressor()))

In [31]:
for model in models:
    steps = []
    steps.append(("Encoder" , Encoder))
    steps.append(("Scaler" , RobustScaler()))
    steps.append(model)
    pipeline = Pipeline(steps=steps)
    scoring = {'r2': make_scorer(r2_score)}
    scores = cross_validate(pipeline, X, y, cv=5, scoring=scoring, return_train_score=True)
    
    print(model[0])
    print("Train R2 Score:", scores["train_r2"].mean())
    print("-" * 10)
    print("Test R2 Score:", scores["test_r2"].mean())
    print("-" * 20)
    print("\n")

RF
Train R2 Score: 0.9560426352365212
----------
Test R2 Score: 0.8033975339155358
--------------------


GB
Train R2 Score: 0.7622270194447238
----------
Test R2 Score: 0.7402301004656785
--------------------


LG
Train R2 Score: 0.8606735447637229
----------
Test R2 Score: 0.8028636055223485
--------------------




In [33]:
from sklearn.model_selection import GridSearchCV

In [69]:
param_grid = {
    'LG__learning_rate': [0.1, 0.01],
    'LG__n_estimators': [400, 600],
    'LG__max_depth': [5, 7]}

In [70]:
steps = []
steps.append(("Encoder" , Encoder))
steps.append(("Scaler" , RobustScaler()))
steps.append(("LG" , lgb.LGBMRegressor()))
pipeline = Pipeline(steps=steps)

In [78]:
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, 
                           scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1, refit=True)

In [79]:
grid_search.fit (X,y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Encoder',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('OHE',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         ['Journey_DayOfWeek',
                                                                          'Source',
                                                                          'Destination']),
                                                                        ('BE',
                                                                         BinaryEncoder(),
                                                                         ['Airline',
                                                                          'Additional_Info'])])),
                                       ('Scaler', RobustScaler()),
                                       ('LG', LGBMRegressor())]),
             n_jobs=-1,
             param_grid={'LG__learning_rate': [0.1, 0.01],
                         'LG__max_depth': [5, 7],
                         'LG__n_estimators': [400, 600]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [80]:
# Print the best parameters and score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", -grid_search.best_score_)

Best Parameters:  {'LG__learning_rate': 0.1, 'LG__max_depth': 7, 'LG__n_estimators': 600}
Best Score:  3732865.8755764244


In [81]:
grid_search.best_params_

{'LG__learning_rate': 0.1, 'LG__max_depth': 7, 'LG__n_estimators': 600}

In [82]:
y_pred = grid_search.best_estimator_.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R2 Score on Validation Set:", r2)

R2 Score on Validation Set: 0.8978184871331811


In [85]:
best_model = grid_search.best_estimator_

In [86]:
import joblib

In [88]:
joblib.dump(final_model , "Model.pkl")
joblib.dump(X.columns , "Inputs.pkl")

['Inputs.pkl']

In [5]:
import joblib

# Load the model
loaded_model = joblib.load("Model.pkl")
loaded_model


Pipeline(steps=[('Encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('OHE',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Journey_DayOfWeek',
                                                   'Source', 'Destination']),
                                                 ('BE', BinaryEncoder(),
                                                  ['Airline',
                                                   'Additional_Info'])])),
                ('Scaler', RobustScaler()),
                ('LG', LGBMRegressor(max_depth=7, n_estimators=600))])

In [89]:
X.columns

Index(['Airline', 'Source', 'Destination', 'Dep_Time', 'Arrival_Time',
       'Duration', 'Total_Stops', 'Additional_Info', 'Journey_Month',
       'Journey_DayOfWeek'],
      dtype='object')

In [6]:
%%writefile app_3rd.py
import streamlit as st
import pandas as pd
import category_encoders
import joblib
import sklearn
import lightgbm as lgb


Inputs = joblib.load("Inputs.pkl")
Model = joblib.load("Model.pkl")

def prediction(Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Journey_Month,Journey_DayOfWeek):
    test_df = pd.DataFrame(columns=Inputs)
    test_df.at[0,"Airline"] = Airline
    test_df.at[0,"Source"] = Source
    test_df.at[0,"Destination"] = Destination
    test_df.at[0,"Dep_Time"] = Dep_Time
    test_df.at[0,"Arrival_Time"] = Arrival_Time
    test_df.at[0,"Duration"] = Duration
    test_df.at[0,"Total_Stops"] = Total_Stops
    test_df.at[0,"Additional_Info"] = Additional_Info
    test_df.at[0,"Journey_Month"] = Journey_Month
    test_df.at[0,"Journey_DayOfWeek"] = Journey_DayOfWeek
    st.dataframe(test_df)
    result = Model.predict(test_df)[0]
    return result

    
def main():
    st.title("flight tickets price prediction")
    Airline = st.selectbox("Airline" , ['Jet Airways','IndiGo', 'Air India', 'Multiple carriers', 'SpiceJet','Vistara',
                                        'Air Asia', 'GoAir','Multiple carriers Premium economy',
                                        'Jet Airways Business','Vistara Premium economy','Trujet'])
    Source = st.selectbox("Source" , ['Delhi','Kolkata','Banglore','Mumbai','Chennai'])
    Destination= st.selectbox("Destination" , ['Cochin','Banglore','Delhi','New Delhi','Hyderabad','Kolkata'])
    Dep_Time = st.slider("Dep_Time" , min_value= 0 , max_value=23 , value=0,step=1)
    Arrival_Time = st.slider("Arrival_Time" , min_value= 0 , max_value=23 , value=0,step=1)
    Duration = st.slider("Duration" , min_value= 0 , max_value=50 , value=0,step= 0.5)
    Total_Stops = st.selectbox("Total_Stops" , [0, 1, 2, 3])
    Additional_Info = st.selectbox("Additional_Info" , ['no info','in-flight meal not included',
                                                        'no check-in baggage included','1 long layover',
                                                        'change airports','business class','1 short layover',
                                                        'red-eye flight','2 long layover'])
    Journey_Month = st.selectbox("Journey_Month" , [3,4,5,6])
    Journey_DayOfWeek = st.selectbox("Journey_DayOfWeek" , ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
    
    if st.button("predict the flight cost"):
        result = prediction(Airline, Source, Destination, Dep_Time, Arrival_Time,Duration, Total_Stops, Additional_Info, Journey_Month,Journey_DayOfWeek)
        st.text(f"The flight ticket will cost {[results]} dollars")
        
if __name__ == '__main__':
    main()    
    

Overwriting app_3rd.py
